## EJECUTA ESTA CELDA PARA AUTENTICARTE CON TU CUENTA DE GMAIL

In [ ]:
from mooclib import *
import inspect, urllib
html, auth_code, userinfo = google_authenticate(PORT_NUMBER=8080)
html

## Ejercicio 1

implementa la siguiente función de coste de una red neuronal de una sola capa:


$$J(\mathbf{b}_1, b_2, \mathbf{W}_1, \mathbf{W}_2) = \frac{1}{m}\sum_{i=0}^{m-1} (\hat{y}-y)^2 + \lambda \left[ || \mathbf{b}_1||_1 + ||\mathbf{W}_1||_1 + ||\mathbf{W}_2||_1\right]$$

donde

$$\hat{y} = g(\mathbf{W}_2 \cdot \tanh(\mathbf{x^T} \cdot \mathbf{W}_1 + \mathbf{b}_1) + b_2)$$

y $||K||_1$ es la norma L1 de $K$, es decir, el número de elementos de $K$ (sea una matriz o un vector) que son distintos de cero.

observaciones:
- puedes usar las funciones `decode`, `y_hat` del ejercicio 2 del problemset 4.
- fíjate que ahora las funciones `decode`, `y_hat` están dentro de `cost` (y no fuera como en el ejercicio referido)
- al incluir la norma L1 en la función de coste, obligamos al minimizador a que penalize a los conjuntos de parámetros con muchos valores distintos de cero. Es decir, favorecemos la dispersión (_sparsity_)

In [ ]:


def cost(X, y, params, reg, nb_cols, h_units):

    def decode(params, nb_cols, h_units):
        ....
        return b1, b2, W1, W2
    
    def y_hat(params, nb_cols, h_units, X):
        return ...

    b1,b2,W1,W2 = decode(params, nb_cols, h_units)

    r = ...
    return r


import urllib, inspect
src1 = urllib.quote_plus(inspect.getsource(cost))

comprueba tu código. el coste más abajo ha de darte 4.807 aprox.

In [ ]:
import numpy as np
def encode(b1, b2, W1, W2): 
    return np.array(list(b1)+[b2]+list(W1.flatten())+list(W2.flatten()))

data = np.loadtxt("data/logregdata.csv", delimiter=",", skiprows=1)
X,y = data[:,:2], data[:,2]

h_units=4
cols = X.shape[1]
b1, b2, W1, W2 = np.ones(h_units), 2, np.ones((cols, h_units))*3, np.ones((h_units,1))*4
b1[[0,2]]=0
W1[:,2]=0
W2[0,0]=0
params = encode(b1,b2,W1,W2)
print "cost     ", cost(X,y,params,0.4, cols, h_units)

#### evaluate your answer before submitting:

In [ ]:
!./run CHECK_SOLUTION QZ2_1 $src1

#### submit your answer (you must be connected to internet)

In [ ]:
!./run SUBMIT_SOLUTION QZ2_1 $src1

## Ejercicio 2

completa la función siguiente para que, dado un estimador, un dataset (X,y) y un array de índices:

1. entrene el estimador con la parte del dataset (X,y) correspondiente a esos índices
2. mida el desempeño (función `score` del estimador) la parte del dataset (X,y) complementaria a esos índices
3. devuelva el desempeño medido.

P.ej. si $X \in \mathbb{R}^{10\times n}$, $y \in \mathbb{R}^{10}$, y `train_indexes=[5,3,0,1,4,2]` el entrenamiento se hará con los primeros seis elementos de $X$ e $y$, y el desemeño (_score_) se medirá con el resto.

In [ ]:
def estimator_train_test(estimator, X, y, train_indexes):
    test_indexes     = ...
    X_train, y_train = ...
    X_test, y_test   = ...
    
    score  = ... 
    return score

import urllib, inspect
src2 = urllib.quote_plus(inspect.getsource(estimator_train_test))

#### evaluate your answer before submitting:

In [ ]:
!./run CHECK_SOLUTION QZ2_2 $src2

#### submit your answer (you must be connected to internet)

In [ ]:
!./run SUBMIT_SOLUTION QZ2_2 $src2

## Ejercicio 3

completa `check_kernel` siguiente para que verifique si una función $K$ es un kernel válido con respecto a una función de transformación $\phi$, dados dos vectores $u$ y $v$.

`check_kernel` tendrá que devolver `True` si:

$$K(u,v) = \langle \phi(u), \phi(v) \rangle$$

donde $\langle \dot \;\;, \dot \; \rangle$ es el producto punto. Revisa las notas de la lección 6 y observa el código de comprobación más abajo para entender cómo debería de comportarse tu función.

**observaciones**

- usa la función [`np.allclose`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.allclose.html) para comparar dos números, mejor que el operador `==` que es sensible a problemas de precisión numérica

In [ ]:
def check_kernel(K_function, phi_function, u, v):
    r = ...
    return r

import urllib, inspect
src3 = urllib.quote_plus(inspect.getsource(check_kernel))

comprueba tu código

In [ ]:
import numpy as np

def kernel(u,v):
    return (u.dot(v)+1)**2

def invalid_kernel(u,v):
    return np.sum(u-v)

def phi(u):
    return np.array([u[0]**2, u[1]**2, np.sqrt(2)*u[0], np.sqrt(2)*u[1], np.sqrt(2)*u[0]*u[1],1 ])

X=np.array([[ 1.5,1.5], [ 1.5,1.], [ 0.,1. ], [ 0.5,1.], [1.,0.5]])

u, v = X[0], X[2]

print "check valid kernel (True expected):    ", check_kernel(kernel,phi, u,v)
print "check NOT valid kernel (False expected):", check_kernel(invalid_kernel,phi, u,v)

#### evaluate your answer before submitting:

In [ ]:
!./run CHECK_SOLUTION QZ2_3 $src3

#### submit your answer (you must be connected to internet)

In [ ]:
!./run SUBMIT_SOLUTION QZ2_3 $src3



## Ejercicio 4

completa la función siguiente para que muestree una partición de train y otra de test a partir de un dataset $(X,y)$, de forma que:

- la parte de test tenga `train_length` datos que deberán de ser **consecutivos** en $(X,y)$
- la parte de test tenga `test_length` datos que deberán de ser **consecutivos** en $(X,y)$ e **inmediatamente** los siguientes a los de train.
- devuelva cuatro variables `X_train, y_train, X_test, y_test` con las particiones correspondientes.

P.ej. para el siguiente dataset:

    X = [[-0.793 -0.094]
         [-0.408  0.811]
         [-0.445  1.143]
         [ 0.15   0.071]
         [-0.438  0.883]
         [-0.736  0.387]
         [-0.834  0.226]
         [ 1.149 -0.554]
         [-0.489  0.387]
         [ 0.361  0.251]]
         
     y = [ 0.  0.  0.  1.  0.  0.  0.  1.  0.  1.]

una posible salida a:

        Xtr, ytr, Xts, yts = resample(X,y,4, 2)
        print Xtr
        print Xts
        print ytr, yts

sería la siguiente:

    [[-0.445  1.143]
     [ 0.15   0.071]
     [-0.438  0.883]
     [-0.736  0.387]]
    [[-0.834  0.226]
     [ 1.149 -0.554]]
    [ 0.  1.  0.  0.] [ 0.  1.]



In [ ]:
def resample(X,y, train_length, test_length):
    
    X_train = ...
    y_train = ...
    X_test  = ...
    y_test  = ...

import urllib, inspect
src4 = urllib.quote_plus(inspect.getsource(resample))

comprueba tu código


In [ ]:
import numpy as np
data = np.loadtxt("data/logregdata.csv", delimiter=",", skiprows=1)
X,y = data[:,:2], data[:,2]
X = X[:10]
y = y[:10]
print "dataset original"
print X
print y

print "\nparticiones train/test"
Xtr, ytr, Xts, yts = resample(X,y,4, 2)
print Xtr
print Xts
print ytr, yts


ids = np.array([np.argwhere(X==i)[0][0] for i in Xtr] + [np.argwhere(X==i)[0][0] for i in Xts])
print ids
np.alltrue((ids - np.min(ids))==np.arange(6))

#### evaluate your answer before submitting:

In [ ]:
!./run CHECK_SOLUTION QZ2_4 $src4

#### submit your answer (you must be connected to internet)

In [ ]:
!./run SUBMIT_SOLUTION QZ2_4 $src4